In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


import nbformat
print(nbformat.__version__)

plotly.offline.init_notebook_mode(connected=True)

5.10.4


In [48]:
import kaggle

kaggle.api.authenticate()

# dataset_files = kaggle.api.dataset_list_files('patrickb1912/ipl-complete-dataset-20082020').files # print the list of files in the dataset

kaggle.api.dataset_download_files('patrickb1912/ipl-complete-dataset-20082020', path='.',unzip=True) # Download the dataset and unzip it in the current directory.

# kaggle.api.dataset_metadata('patrickb1912/ipl-complete-dataset-20082020', path='.') # Download metadata of the dataset
# datasets = kaggle.api.dataset_list(search='ipl')

Dataset URL: https://www.kaggle.com/datasets/patrickb1912/ipl-complete-dataset-20082020


In [49]:
Df_deliveries = pd.read_csv(r'F:\DataAnalysis\IPL_project\deliveries.csv')
Df_matches= pd.read_csv(r'F:\DataAnalysis\IPL_project\matches.csv')

In [50]:
Df_matches['date'] = pd.to_datetime(Df_matches['date'])
Df_matches = Df_matches.replace({
    '2009/10':'2010'
})
Df_matches['season'] = Df_matches['season'].apply(lambda x : x[:-3] if x.__contains__('/') else x)

Df_matches['city'] = Df_matches['city'].fillna('International')
def interchange(row):
    if row['toss_decision'] == 'field':
        if row['toss_winner'] != row['team2']:
            # Swap team1 and team2
            row['team1'], row['team2'] = row['team2'], row['team1']
    else:
        if row['toss_winner'] != row['team1']:
             # Swap team1 and team2
            row['team1'], row['team2'] = row['team2'], row['team1']
    return row


Df_matches = Df_matches.apply(interchange,axis=1)



#### Standardizing Venue Teams and winners


In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import numpy as np

colSta = ['venue','team1','team2','toss_winner','winner']

for each in colSta:
    Df_matches[each] = Df_matches[each].fillna('Match Stopped')
    vectorize = TfidfVectorizer()
    x = vectorize.fit_transform(Df_matches[each])

    dbscan = DBSCAN(eps=0.5,min_samples=5,metric='cosine')
    labels = dbscan.fit_predict(x)

    Df_matches['Cluster'] = labels


    conical_name = (Df_matches.groupby('Cluster')[each].apply(lambda venue : sorted(venue,key=len)[0]).to_dict())
    conical_name

    Df_matches[each] = Df_matches['Cluster'].map(conical_name)

    Df_matches.drop(columns='Cluster', inplace=True)


In [52]:
Df_matches = Df_matches.replace({
    'Delhi Daredevils':'Delhi Capitals',
    'Gujarat Lions':'Gujarat Titans',
    'Royal Challengers Bangalore':'Royal Challengers Bengaluru',
    'Rising Pune Supergiants':'Rising Pune Supergiant',
    'Kings XI Punjab':'Punjab Kings'
})

In [53]:
import pickle

with open('Df_matches.pkl','wb') as f:
    pickle.dump(Df_matches,f)


# season wise match Count

In [54]:
Season_matches = Df_matches['season'].value_counts().to_frame()
Season_matches = Season_matches.sort_index(ascending=False)
fig = px.bar(Season_matches,orientation='h',text_auto='f')
fig.update_layout(title={
        'text': "IPL Season's with Total Matches",  # Title text
        'y': 0.95,  # Position of title vertically
        'x': 0.5,  # Position of title horizontally (centered)
        'xanchor': 'center',  # Anchoring for horizontal position
        'yanchor': 'top',  # Anchoring for vertical position
        'font': {'size': 25, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}  # Bold and styled font
    },
    showlegend=False,
    xaxis=dict(
        ticks='outside',

),
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',
)

fig.update_xaxes(
    title="No.of Matches",
    showticklabels=False,
    ticks='',
)

fig.update_yaxes(
    title="Seasons"
)

fig.update_traces(textfont_size=14,textangle=0, textposition="outside", cliponaxis=False)
fig.show()


# Most matches hosted city

In [55]:
city_matches = Df_matches['city'].value_counts().to_frame()

city_matches = city_matches.reset_index(names='match Count')

fig = px.bar(city_matches.head(10),text_auto='0.2s',y='count',x='match Count',labels={'match Count':'Hosted City','count':'No.of Matches'})

fig.update_layout(title={
    'text':"City hosted the most matches",
    'x':0.5,
    'y':0.95,
    'xanchor':'center',
    'yanchor':'top',
    'font':{
        'size':25,
        'weight':'bold'
    }
},showlegend=False,
plot_bgcolor='white')

fig.show()

#### Most utilized Venues

In [71]:
venue_matches = Df_matches['venue'].value_counts().to_frame()

venue_matches = venue_matches.reset_index(names='match Count')

fig = px.bar(venue_matches.head(10),text_auto='0.2s',y='count',x='match Count',labels={'match Count':'Hosted Venue','count':'No.of Matches'})

fig.update_layout(title={
    'text':"Stadium hosted the most matches",
    'x':0.5,
    'y':0.95,
    'xanchor':'center',
    'yanchor':'top',
    'font':{
        'size':25,
        'weight':'bold'
    }
},showlegend=False,
plot_bgcolor='white')

fig.show()

In [57]:
# help(fig.up)
hignManofMatch = Df_matches['player_of_match'].value_counts().to_frame()
fig = px.bar(hignManofMatch.head(10)[::-1],orientation='h',text_auto='0.2s')
fig.update_layout(title = {
    'text': "Top 10 Players of the Match",
    'x': 0.5,
    'y': 0.95,
    'xanchor': 'center',
    'yanchor': 'top',
    'font':{
        'size':25,
        'weight':'bold'
    }
},
    showlegend=False,
    plot_bgcolor='white')

fig.update_xaxes(title="No. of time wonned Man of the Match",ticks="")#, titlefont_size=16, tickfont_size=14)
fig.update_yaxes(title="Top 10 Players")#, titlefont_size=16, tickfont_size=14)

### Create Team Data Frame

In [58]:
TeamDF = Df_matches['team1'].value_counts() + Df_matches['team2'].value_counts()

TeamDF = TeamDF.to_frame()
TeamDF = TeamDF.reset_index(names='Team')
# TeamDF.set_index('Team',inplace=True)
TeamDF.rename(columns={'count':'matches_count'},inplace=True)

TeamDF

,Team,matches_count
0,Chennai Super Kings,238
1,Deccan Chargers,75
2,Delhi Capitals,252
3,Gujarat Titans,75
4,Kochi Tuskers Kerala,14
5,Kolkata Knight Riders,251
6,Lucknow Super Giants,44
7,Mumbai Indians,261
8,Pune Warriors,46
9,Punjab Kings,246


In [59]:
winTeam = Df_matches['winner'].value_counts().to_frame()
winTeam = winTeam.reset_index(names='Team')
winTeam
teamstat = pd.merge(TeamDF,winTeam,on='Team',how='right')
teamstat.rename(columns={'count':'win_count'},inplace=True)
teamstat = teamstat.dropna()


In [60]:
teamstat['matches_count'] = teamstat['matches_count'].apply(lambda x: int(x))
teamstat['loss_count'] = teamstat['matches_count'] - teamstat['win_count']
teamstat['win_perc'] =  (teamstat['win_count']/teamstat['matches_count'])*100

In [61]:
teamstat = teamstat.sort_values(by='matches_count',ascending=False)
teamstat
fig = px.bar(teamstat.reset_index(),orientation='v',x='Team',y=['win_count','loss_count'],text_auto='0s',labels={
    'Team':'IPL Teams',
    'value': 'Total Matches Played'
},hover_name='Team')

fig.update_layout(xaxis=dict(
            tickangle=25 # Rotate x-axis labels by 45 degrees
        ),showlegend=False,title=dict(
            text="Total Matches Played by Each Team in IPL",
            x=0.5,
            y=0.95,
            font=dict(size=20, color='black',weight='bold'),
        ),
        plot_bgcolor='white')

fig.show()

# Match win percentage

In [62]:
perc = teamstat[teamstat['matches_count'] > 100].sort_values(by='win_perc',ascending=False)


fig = px.bar(perc[::-1],y='Team',x='win_perc',orientation='h',text_auto='0.2s',hover_data='matches_count')

fig.update_layout(xaxis=dict(
        ),showlegend=False,title=dict(
            text="Win Convertion of Teams",
            x=0.5,
            y=0.95,
            font=dict(size=20, color='black',weight='bold'),
        ),
        plot_bgcolor='white')
fig.show()


In [63]:
Df_playoff = Df_matches[Df_matches['match_type'] != 'League'].copy()

Df_playoff['TeamComp'] = Df_playoff.apply(lambda row : [row['team1'],row['team2']],axis=1)

Df_playoff_exploded = Df_playoff.explode('TeamComp')

In [64]:
playoff = Df_playoff_exploded.pivot_table(index='TeamComp',aggfunc='size',columns='season').apply(lambda x : x/x )
playoff = playoff.fillna(0)
playoff['playoff_count'] = playoff.sum(axis=1)
playoff = playoff.sort_values(by='playoff_count',ascending=True)

fig = px.bar(playoff.reset_index(),x='playoff_count',y='TeamComp',labels={
    'TeamComp':'Teams in Playoff',
    'playoff_count':'Number of Playoff'
})

fig.update_layout(title=dict(
    text="Teams appearance in Playoff",
    x=0.5,
    y=0.95,
    font=dict(weight='bold',size=20)
),
plot_bgcolor='white')

fig.show()

In [65]:
Df_matches_league = Df_matches[Df_matches['match_type'] == 'League']
match_pivot=Df_matches_league.pivot_table(columns='winner',index='season',aggfunc='size',fill_value=0)
match_pivot.loc['Total'] = match_pivot.sum()
# match_pivot['Total Matces'] = match_pivot.sum(axis=True)
match_pivot = match_pivot[match_pivot.loc['Total'].sort_values(ascending=False).index]
match_pivot.drop('Total',inplace=True)
match_pivot = match_pivot.div(14/100)
match_pivot
plotable = match_pivot
plotable = plotable.reset_index()
plotable.columns

fig = px.line(plotable,x='season',y=plotable.columns[4],line_shape='spline',labels={'season':'Seasons (Years)','value':'Match winning in %'})

Title = 'Team Performance in league'

fig.update_layout(showlegend=True,title=dict(
            text=Title,
            x=0.5,
            y=0.95,
            font=dict(size=20, color='black',weight='bold'),
        ),
        plot_bgcolor='white')


fig.update_xaxes(gridcolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey')


fig.show()

In [66]:
def get_matchwin(Df,team1):

    Df = Df[(Df['team1'] == team1) | (Df['team2'] == team1)]

    opponents = set(Df['team1'].tolist() + Df['team2'].tolist())
    opponents.discard(team1)

    result = []

    for opponent in opponents:
        matchwin = Df[((Df['team1'] == team1) & (Df['team2'] == opponent)) | ((Df['team1'] == opponent) & (Df['team2'] == team1))]

        winlist = matchwin['winner'].value_counts().to_dict()
        
        result.append({'Team': team1 , 'Opponent': opponent, 'Wins': winlist.get(team1,0), 'Opponent Wins': winlist.get(opponent,0)})

    return pd.DataFrame(result)


In [67]:
Df_vs = Df_matches.copy()
Team = 'Punjab Kings'
Df_vs = get_matchwin(Df_vs,Team)
Df_vs['Team win Percentage'] = Df_vs['Wins']/(Df_vs['Wins'] + Df_vs['Opponent Wins'])*100
Df_vs['Opponent win Percentage'] = Df_vs['Opponent Wins']/(Df_vs['Wins'] + Df_vs['Opponent Wins'])*100
Df_vs = Df_vs.sort_values(by='Team win Percentage',ascending=True)

In [68]:
fig = px.bar(Df_vs,x=['Team win Percentage','Opponent win Percentage'], y='Opponent',text_auto='0.2s',labels={'Opponent':'Opponent Teams','value':f'{Team} Win Ratio in % '},
             hover_data=['Wins','Opponent Wins'])
fig.update_layout(plot_bgcolor="white",yaxis=dict(side='right'),legend=dict(title=f'{Team} Win Distribution',y=1,x=0.1,xanchor='center',yanchor='bottom'), title=dict(text=Team,x=0.5,y=0.95,font=dict(weight='bold')))
fig.update_traces(textposition='inside')
fig.show()

In [69]:
# Create the pivot table and process it
Tosswinner = Df_matches.pivot_table(index=['toss_winner', 'toss_decision'], aggfunc='size')
Tosswinner = Tosswinner.reset_index(name='count')
Tosswinner[Tosswinner['toss_winner'] == 'Chennai Super Kings']

fig = make_subplots(rows=2,cols=7,
    specs=[[{'type': 'domain'}] * 7, [{'type': 'domain'}] * 7],  # Define domain for pie charts
    subplot_titles=list(set(Tosswinner['toss_winner'])))

teams = list(set(Tosswinner['toss_winner']))


for i, team in enumerate(teams,start=1):
    x = 1
    y = i
    if i > 7:
        y = i-7
        x = 2
    v = Tosswinner[Tosswinner['toss_winner'] == team]
    fig.add_trace(go.Pie(values=v['count'],labels=v['toss_decision'],hovertext=team),x,y)


fig.update_layout(
    height=600,
    width=1600,  # Adjust height dynamically based on the number of teams
    title= dict(text="Toss Decision Distribution by Team",font=dict(weight='bold',size=20)),
    title_x=0.5,

)

fig.show()

### Teams Runs and Wicket Distribution

In [70]:
Df_Runs= Df_matches[Df_matches['result'] ==  'runs'] # runs wickets
Df_Wickets= Df_matches[Df_matches['result'] ==  'wickets']

fig = make_subplots(rows=2,cols=2,
    specs=[[{'colspan':2},None],
           [{},{}]],
    subplot_titles=('Target Plot Distribution','Result Margin by Wicket Distribution','Result Margin by Run Distribution')
    )

fig.add_trace(go.Histogram(x=Df_matches['target_runs'],nbinsx=25),row=1,col=1)
fig.add_trace(go.Histogram(x=Df_Wickets['result_margin'],nbinsx=25),row=2,col=1)
fig.add_trace(go.Histogram(x=Df_Runs['result_margin'],nbinsx=50),row=2,col=2)
# fig = px.histogram(Df_matches,x='target_runs',nbins=25)

fig.update_layout(showlegend=False,height=600,plot_bgcolor='white')
fig.show()